In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 72.8 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 27.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-10-28 16:17:35.249262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761668255.719450      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761668255.830068      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
def generate_text(prompt, max_length=100, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [5]:
!pip install sentence-transformers PyPDF2 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 47.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━

In [6]:
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
import faiss

In [7]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() + "\n"
    return full_text

In [8]:
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [9]:
def embed_chunks(chunks, model_name='sentence-transformers/all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks, convert_to_numpy=True)
    return model, embeddings

In [10]:
def create_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

In [11]:
def search_index(query, model, index, chunks, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

In [12]:
pdf_path = "/kaggle/input/tipshindawi-university/Tips Hindawi University Info.pdf"  

text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(text,chunk_size=50, overlap=5)

model_embeddings, embeddings = embed_chunks(chunks)


index = create_faiss_index(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
question = "Where is Tips Hindawi University located?"
top_chunks = search_index(question, model_embeddings, index, chunks, k=3)

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---\n{chunk}")

chunk = top_chunks[0]
prompt = f"Answer the next question: {question} by reading the following text:{chunk}"
answer = generate_text(prompt, max_length=500)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Chunk 1 ---
1. General Overview Tips Hindawi University (THU) is a premier institution of higher education located in the heart of the Middle East. Founded in 1963, the university has grown into a globally recognized center for academic excellence and innovation. With over six decades of educational leadership, THU has produced more

--- Chunk 2 ---
leadership, THU has produced more than 150,000 graduates who serve in diverse industries and academic circles worldwide. The university is accredited by the International Commission for Academic Standards and the Ministry of Higher Education. It operates under the guiding motto: "Knowledge, Integrity, Progress" , and fosters an environment of research,

--- Chunk 3 ---
Four residence halls - A medical center - Sports complex and stadium - Innovation and entrepreneurship hub 2.2 Satellite Campuses North Campus : Focused on agricultural sciences and environmental research West Campus : Home to the School of Design and Architecture 2.3 St

In [22]:
print(answer[0])

Answer the next question: Where is Tips Hindawi University located? by reading the following text:1. General Overview Tips Hindawi University (THU) is a premier institution of higher education located in the heart of the Middle East. Founded in 1963, the university has grown into a globally recognized center for academic excellence and innovation. With over six decades of educational leadership, THU has produced more than 150,000 graduates who have gone on to make significant contributions to their respective fields and communities.

2. Campus and Location
   - **Main Campus**: The main campus of Tips Hindawi University is situated in the bustling city of Dubai, United Arab Emirates. Spread over an expansive area of 1.3 million square meters, the campus offers a vibrant and multicultural learning environment. It is home to state-of-the-art facilities, including academic buildings, research centers, student accommodations, and recreational spaces.
   - **Additional Campuses**: In additi

In [23]:
question = "Does the university offer online programs?"
top_chunks = search_index(question, model_embeddings, index, chunks, k=3)

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---\n{chunk}")

chunk = top_chunks[0]
prompt = f"Answer the next question: {question} by reading the following text:{chunk}"
answer = generate_text(prompt, max_length=500)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Chunk 1 ---
leadership, THU has produced more than 150,000 graduates who serve in diverse industries and academic circles worldwide. The university is accredited by the International Commission for Academic Standards and the Ministry of Higher Education. It operates under the guiding motto: "Knowledge, Integrity, Progress" , and fosters an environment of research,

--- Chunk 2 ---
Graduate Programs MBA (Master of Business Administration) MSc in Data Science MA in Psychology PhD in Mechanical Engineering PhD in Political Science 4. Admissions and Tuition 4.1 Undergraduate Admissions High school GPA of 85% or equivalent English proficiency test (IELTS 6.0 or TOEFL 80) Entrance interview for certain programs 4.2

--- Chunk 3 ---
1. General Overview Tips Hindawi University (THU) is a premier institution of higher education located in the heart of the Middle East. Founded in 1963, the university has grown into a globally recognized center for academic excellence and innovation. With ov

In [24]:
print(answer[0])

Answer the next question: Does the university offer online programs? by reading the following text:leadership, THU has produced more than 150,000 graduates who serve in diverse industries and academic circles worldwide. The university is accredited by the International Commission for Academic Standards and the Ministry of Higher Education. It operates under the guiding motto: "Knowledge, Integrity, Progress" , and fosters an environment of research, innovation, and continuous learning. THU also provides a wide range of online programs, allowing students to pursue their academic goals from anywhere in the world.

Yes, the university offers online programs.


In [25]:
question = "Is there financial aid for international students?"
top_chunks = search_index(question, model_embeddings, index, chunks, k=3)

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---\n{chunk}")

chunk = top_chunks[0]
prompt = f"Answer the next question: {question} by reading the following text:{chunk}"
answer = generate_text(prompt, max_length=500)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Chunk 1 ---
interview for certain programs 4.2 Graduate Admissions Relevant bachelor’s degree Minimum GPA of 3.0/4.0 Two academic references Statement of purpose 4.3 Tuition Fees (Annual) Undergraduate: $5,000 - $8,000 Graduate: $6,500 - $10,000 4.4 Scholarships Merit Scholarships : 25% to 100% tuition coverage Need-Based Grants Research Fellowships for graduate students 5.

--- Chunk 2 ---
Graduate Programs MBA (Master of Business Administration) MSc in Data Science MA in Psychology PhD in Mechanical Engineering PhD in Political Science 4. Admissions and Tuition 4.1 Undergraduate Admissions High school GPA of 85% or equivalent English proficiency test (IELTS 6.0 or TOEFL 80) Entrance interview for certain programs 4.2

--- Chunk 3 ---
Fellowships for graduate students 5. Administration President : Dr . Nabil Al-Khatib• • • • • • • • • • • • • • • • • • • • • • • 2 Vice President of Academic Affairs : Prof. Layla Mahmoud Dean of Students : Mr .


In [26]:
print(answer[0])

Answer the next question: Is there financial aid for international students? by reading the following text:interview for certain programs 4.2 Graduate Admissions Relevant bachelor’s degree Minimum GPA of 3.0/4.0 Two academic references Statement of purpose 4.3 Tuition Fees (Annual) Undergraduate: $5,000 - $8,000 Graduate: $6,500 - $10,000 4.4 Scholarships Merit Scholarships : 25% to 100% tuition coverage Need-Based Grants Research Fellowships for graduate students 5.1 International Students The university is authorized to issue I-20 form for F-1 visa. I-20 form is required for international students to apply for an F-1 visa. I-20 form is issued after acceptance into the university. International students must provide proof of financial support. International students must maintain a full-time status during their studies. 5.2 Financial Aid for International Students International students are not eligible for federal financial aid. However, they may be eligible for private loans, schola

In [29]:
question = "What languages are used for instruction?"
top_chunks = search_index(question, model_embeddings, index, chunks, k=3)

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---\n{chunk}")

chunk = top_chunks[0]
prompt = f"Answer the next question: {question} by reading the following text:{chunk}"
answer = generate_text(prompt, max_length=700)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Chunk 1 ---
Events and Traditions Founders' Week Annual Cultural Festival Spring Research Showcase 7.3 Support Services Mental Health Counseling Career Development Center Tutoring and Writing Labs Disability Services 8. Faculty Highlights Dr. Yasir Al-Sabbagh (Computer Science): Expert in natural language processing Prof. Rana Khalil (Economics): World Bank consultant and author Dr. Omar

--- Chunk 2 ---
Dormitory (First-year undergraduates) Rafidain Apartments (Graduate students) Amal Housing Complex (Family and international students) 3. Academic Structure 3.1 Faculties Faculty of Engineering Faculty of Medicine and Health Sciences Faculty of Business and Economics Faculty of Arts and Humanities Faculty of Law and International Studies Faculty of Computer and Information Sciences

--- Chunk 3 ---
of Computer and Information Sciences Faculty of Architecture and Design• • • • • • • • • • • • 1 3.2 Sample Undergraduate Programs BSc in Computer Science BA in International Relations 

In [30]:
print(answer[0])

Answer the next question: What languages are used for instruction? by reading the following text:Events and Traditions Founders' Week Annual Cultural Festival Spring Research Showcase 7.3 Support Services Mental Health Counseling Career Development Center Tutoring and Writing Labs Disability Services 8. Faculty Highlights Dr. Yasir Al-Sabbagh (Computer Science): Expert in natural language processing Prof. Rana Khalil (Economics): World Bank consultant and author Dr. Omar Al-Kaysi (Biology): Specialist in molecular biology and genetics Dr. Lina Saad (Psychology): Specialist in mental health and well-being Dr. Ali Al-Fayez (History): Specialist in Middle Eastern history and culture Dr. Nabeel Al-Sayed (Engineering): Expert in sustainable energy systems

The languages used for instruction at the university are English and Arabic.
